In [1]:
! pip install langchain-community faiss-cpu

  Using cached faiss_cpu-1.11.0.post1-cp311-cp311-win_amd64.whl (14.9 MB)



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello Words")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [7]:
# Manage Vecotr Store

from uuid import uuid4
from langchain_core.documents import Document
document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

ids=[str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents,ids=ids)


['63c6a0f0-4273-4c5a-9e7a-8f9bb7e553a2',
 '15a99562-0f7a-4097-8595-995a57071669',
 '1a426405-6ae9-47fd-8343-b3b815fb5953',
 '74dbea65-7a9e-4a91-91d2-ff6a22a80cec',
 'c50d8560-ff89-441c-8b48-88f0d0e3d246',
 '6966c836-96a4-47ec-a215-cbfd1bfa94c1',
 '291464d0-fff6-4b75-a996-33cb9b9b21ec',
 'aaee1519-509f-4eb3-9d5c-a3ee0fd7882b',
 '59f57ca9-8681-46df-9277-781207ec66dc',
 'abc4b437-ff37-4cbb-bbe9-429596930a94']

In [10]:
results=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,filter={"source":"tweet"}
)
for res in results:
    print(f"**{res.page_content} [{res.metadata}]")

**Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
**LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [11]:
results=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,filter={"source":{"$eq":"tweet"}}
)
for res in results:
    print(f"**{res.page_content} [{res.metadata}]")

**Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
**LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [13]:
# search with score

results=vector_store.similarity_search_with_score(
     "LangChain provides abstractions to make working with LLMs easy",
     k=1,
     filter={"source":"tweet"}
)
for res,score in results:
    print(f"**[SIM={score:3f}] {res.page_content} [{res.metadata}]")

**[SIM=1.044638] Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


C:\Users\malik\AppData\Local\Temp\ipykernel_8032\2453200606.py:3: RuntimeWarning: coroutine 'FAISS.asimilarity_search_with_score' was never awaited
  results=vector_store.similarity_search_with_score(


In [14]:
retriever=vector_store.as_retriever(search_type="mmr",search_kwargs={"k":2})
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='74dbea65-7a9e-4a91-91d2-ff6a22a80cec', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(id='59f57ca9-8681-46df-9277-781207ec66dc', metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.')]

In [15]:
vector_store.save_local('faiss_index')
new_vector_store=FAISS.load_local(
    "faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True
)
docs=new_vector_store.similarity_search('qux')

In [16]:
docs

[Document(id='1a426405-6ae9-47fd-8343-b3b815fb5953', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='aaee1519-509f-4eb3-9d5c-a3ee0fd7882b', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='c50d8560-ff89-441c-8b48-88f0d0e3d246', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='291464d0-fff6-4b75-a996-33cb9b9b21ec', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]